<p style="font-size:32px; font-weight: bolder; text-align: center"> Ring Polymer Molecular Dynamics </p>

This notebook provides a hands-on counterpart to the "Ring polymer molecular dynamics" lecture for the MOOC "Path Integrals in Atomistic Modeling". 

The different sections in this notebook match the parts this lecture is divided into:

1. RPMD 2
2. RPMD 3 
3. RPMD 4 

<p style="color:blue; font-weight:bold"> Questions in blue invite you to reflect on the results you are obtaining. If you are doin </p>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ase, ase.io
import chemiscope
import pimdmooc
pimdmooc.add_ipi_paths()

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
# It's also possible to use the reduced notation by directly setting font.family:
plt.rcParams.update({
  "text.usetex": False,
  "font.family": "Helvetica"
})

# EXERCISE 1: Quantum time correlation functions

We consider a (classical) harmonic oscillator with frequency $\omega$ and unit mass, in the constant-temperature ensemble at inverse temperature $\beta$. 

## Standard quantum time correlation function

The standard quantum time auto-correlation function of the position operator is given as

$C^{\text{std}}_{\hat{q}\hat{q}} = Z^{-1} \sum_{i} e^{-\beta E_{i}} \sum_{j} e^{ -i(E_{i} - E_{j}) t / \hbar} ~ <i| \hat{q} | j> ~ < j | \hat{q} | i >$

where 
- $E_i$ is the energy of the i-th eigenstate of the system
- $\langle i|\hat{q}|j\rangle$ is the ij-th matrix element of the operator q i.e. <i | q| j>
- $<j|\hat{q}|i>$ is the ji-th matrix element of the operator q i.e. <j | q| i>
- $Z$ is the canonical partition function

Let us define functions for some of these for a simple harmonic oscillator [reduced  units]!

In [ ]:
m = 1
hbar = 1
beta = 1

In [ ]:
def Ei(i, omega=1, return_zero=False):
    """
    Return the energy of the i-th eigen state of the simple harmonic oscillator.
    """
    
    if return_zero:
        return (i + 1/2) * hbar * omega 
    else:
        return (i) * hbar * omega 
    
    
def qij(i, j, omega=1):
    """
    Return the expection value of <i| q |j> where q is the position operator and 
    |i>, |j> correspond to the i-th and j-th eigenstate of the sinple harmonic oscillator.
    Invoking ladder operators, the position  operator is written as
    
    q = \sqrt[ h / (2 m w) ] (a + a†) 
    
    Thus, 
    
    qij = \sqrt[ h / (2 m w) ] (<i | a | j> + <i | a† | j>)
    
    with
    
    <i | a | j> = sqrt[j] delta(i, j-1)
    <i | a† | j> = sqrt[j+1] delta(i, j+1)
    
    """
    
    if i == j - 1:
        return np.sqrt(hbar / (2 * m * omega)) * np.sqrt(j) 
    elif i == j + 1:
        return np.sqrt(hbar / (2 * m * omega)) * np.sqrt(j + 1) 
    else: 
        return 0

Let us now define a function that evaluates the standard time correlation function by summing up matrix elements up to the i-th eigenstate!

In [ ]:
def Cqq_std(t, omega=1, i_max=20):
    """
    Calculates the standard quantum time correlation function of a simple harmonic oscillator 
    by considering the first i_max eigenstates.
    """
    
    rc = 0
    rz = 0
        
    for i in range(i_max + 1):
        
        sumj = 0
        for j in range(0, i + 2): 
            sumj += np.exp(-1j * (Ei(i) - Ei(j)) * t / hbar) * qij(i,j) * qij(j,i)
            
        rc += sumj * np.exp (-beta * Ei(i))
        rz += np.exp (-beta * Ei(i))

    
    if i_max > 0:
        return rc / rz
    else:
        return t * 0

# Let us first converge the standard time correlation function w.r.t the number of eigenstates!

In [ ]:
t = np.linspace(-10,10, 100)
i_max = 10

plt.title('Convergence of Re' + r'($C^{\mathrm{std}}_{qq}$)' + ' with the number of eigenstates')
plt.xlabel('time [a.u.]')
plt.ylabel('Re($C^{\mathrm{std}}_{qq}$) [a.u.]')
for i in range(2, i_max, 2):
    plt.plot(t, np.real(Cqq_std(t, 1, i)), label= 'up to ' +  str(i) + '-th state', ls='--')
plt.legend()
plt.show()


plt.title('Convergence of Im($C_{qq}$) with the number of eigenstates')
plt.xlabel('time [a.u.]')
plt.ylabel('Im($C^{\mathrm{std}}_{qq}$) [a.u.]')
for i in range(2, i_max, 2):
    plt.plot(t, np.imag(Cqq_std(t, 1, i)), label= 'up to ' +  str(i) + '-th state', ls='--')
plt.legend()
plt.show()



Can you now plot the "convered" position autcorrelation function of a simple harmonic oscillator?

In [ ]:
t = np.linspace(-10,10, 100)
plt.title(r'Converged $C_{qq}$ of a harmonic oscillator at $\beta$ = %5.2e' % (beta))
plt.xlabel('time [a.u.]')
plt.ylabel('$C^{\mathrm{std}}_{qq}$ [a.u.]')
plt.plot(t, np.real(Cqq_std(t)), label='Real', marker='o')
plt.plot(t, np.imag(Cqq_std(t)), label='Imaginary', marker='o')
plt.legend()
plt.show()

## Kubo-transformed time correlation function

The Kubo-transformed quantum time auto-correlation function of the position operator is given as

$C^{\text{kubo}}_{\hat{q}\hat{q}} = (Z \beta)^{-1} \int_{0}^{\beta} d\lambda  \sum_{i} e^{-(\beta - \lambda) E_i} 
   (\sum_{j} exp^{-\lambda E_j} \exp^{ -i(E_i - E_j) t / \hbar} ~ <i| \hat{q} | j> ~ < j | \hat{q} | i >)$

which can also be written as

$\bar{C}_qq = Z^{-1} \sum_{i} e^{-\beta E_i} \sum_{j} e^{ -i(E_i - E_j) t / \hbar}  ~ <i| \hat{q} | j> ~ < j | \hat{q} | i > 
              \frac{\int_{0}^{\beta} d\lambda  e^{-\lambda (E_j-E_i)}}{\int_{0}^{\beta} d\lambda} $



Let us reuse the functions to study the convergence of the Kubo-transformated position auto-correlation function. 
Note that we will need to converge w.r.t the granuality of the integration over imaginary time (beta) and over the maximum occupied eigenstate. To begin, we can assume the two to be independent parameters.

In [ ]:
def Cqq_kubo(t, omega=1, i_max=20, lgrid=1000):
    """
    Calculates the standard quantum time correlation function of a simple harmonic oscillator 
    by considering the first i_max eigenstates.
    """
    
    rcqq = 0
    rz = 0
    
    lgrid = np.linspace(0, beta, lgrid)
    
    for i in range(i_max + 1):
                
        for j in range(i + 2): 
            
            lterm = np.sum(np.exp(-lgrid * (Ei(j) - Ei(i)))) * (lgrid[1] - lgrid[0]) / beta
            rcqq += lterm * np.exp (-(beta) * Ei(i)) * np.exp(-1j * (Ei(i) - Ei(j)) * t / hbar) * qij(i,j) * qij(j,i)
            
        rz += np.exp (-beta * Ei(i))
        
    if i_max > 0:
        return rcqq / rz
    else:
        return t * 0

Let's look at the convergence w.r.t the number of occupied eigenstates. You will notice that the imaginary part of the Kubo transformed time correlation function will go to zero (as it should ;)).

In [ ]:
t = np.linspace(0,20, 1000)
i_max =10

plt.title('Convergence of Re(${C}_{qq}$) with the number of eigenstates')
plt.xlabel('time [a.u.]')
#plt.ylabel('Re($C_{qq}$) [a.u.]')
for i in range(2, i_max, 2):
    plt.plot(t, np.real(Cqq_kubo(t, 1, i)), label= 'up to ' +  str(i) + '-th state', ls='--')
plt.legend()
plt.show()


plt.title('Convergence of Im($C_{qq}$) with the number of eigenstates')
plt.xlabel('time [a.u.]')
plt.ylabel('Im($C_{qq}$) [a.u.]')
for i in range(2, i_max, 2):
    plt.plot(t, np.imag(Cqq_kubo(t, 1, i)), label= 'up to ' +  str(i) + '-th state', ls='--')
plt.legend()
plt.show()



In [ ]:
lgrid = np.logspace(1,5, 5)
i_max =20

plt.title('Convergence of Re(${C}_{qq}$) with the quadrature of the $\lambda$ integral')
plt.xlabel('time [a.u.]')
for l in lgrid:
    l = int(l)
    plt.plot(t, np.real(Cqq_kubo(t, 1, i_max, l)), label= 'up to ' +  str(l), ls='--')
plt.legend()
plt.show()


Lets simultaneously plot the converged real and imaginary components of the Kubo transformed time correlation function!

In [ ]:
t = np.linspace(-10,10, 100)
plt.title('Converged $C_{qq}$ of a harmonic oscillator\n [m = 1; hbar = 1; $beta$ = 1; $\omega$=1]')
plt.xlabel('time [a.u.]')
plt.ylabel('$C_{qq}$ [a.u.]')
plt.plot(t, np.real(Cqq_kubo(t)), label='Real', marker='o')
plt.plot(t, np.imag(Cqq_kubo(t)), label='Imaginary', marker='o')
plt.legend()
plt.show()

## Detailed balance relation

As you can see for $\beta$ >> $\hbar \omega$ the standard and the kubo-transformed quantum position autocorrelation functions deviate from one another. Let us now verify whether their Fourier transforms are related by the harmonic detail balance correction factor.

In [ ]:
t = np.linspace(-100,100, 10000)
plt.title('$C_{qq}$ of a harmonic oscillator\n [m = 1; hbar = 1; $beta$ = 1; $\omega$=1]')
plt.xlabel('time [a.u.]')
plt.ylabel('$C_{qq}$ [a.u.]')
plt.plot(t, np.real(Cqq_std(t)), label='std')
plt.plot(t, np.real(Cqq_kubo(t)), label='kubo')
plt.legend()
plt.show()


To ensure that we can verify this relation numerically, we multiply the position autocorrelation by a soft damping function. This ensures that our signal goes to zero and thus its integral can numerically computed by quadrature. In practice this prevents spectral leakage. 

In [ ]:
t = np.linspace(-200,200, 10000)

fdamp = np.exp(-(t / 20 / np.pi)**2)


plt.title('Damped $C_{qq}$ of a harmonic oscillator\n [m = 1; hbar = 1; $beta$ = 1; $\omega$=1]')
plt.xlabel('time [a.u.]')
plt.ylabel('$C_{qq}$ [a.u.]')
plt.plot(t, np.real(Cqq_std(t) * fdamp), label='std')
plt.plot(t, np.real(Cqq_kubo(t) * fdamp), label='kubo')
plt.legend()
plt.show()


Let's numerically calculate the left and right hand sides of Eq. 4.11, i.e. compute the Fourier transforms of the standard and Kubo transformed autocorrelation functions and see if their ratio is given by Eq. 4.12.

In [ ]:
# frequency grid 
w = np.linspace(0, 3, 300)

# time grid
t = np.linspace(-200,200, 10000)
dt = t[1] - t[0]

# damping function to prevent spectral leakage
fdamp = np.exp(-(t / 20 / np.pi)**2)

# calculates the Fourier transform of the std quantum time correlation function
cqq_std = Cqq_std(t) * fdamp
kqq_std = np.zeros(w.shape)
for i in range(len(w)):
    kqq_std[i] = np.real(np.sum(dt * np.exp(-1j * w[i] * t) * cqq_std))

# the harmonic detail balance correction factor
dbcf = np.zeros(w.shape)
dbcf[1:]  = (1 - np.exp ( - beta * hbar * w[1:])) / ( beta * hbar * w[1:])

kqq_times_dbcf = kqq_std * dbcf

# calculates the Fourier transform of the jubo-transformed quantum time correlation function
cqq_kubo = Cqq_kubo(t) * fdamp
kqq_kubo = np.zeros(w.shape)
for i in range(len(w)):
    kqq_kubo[i] = np.real(np.sum(dt * np.exp(-1j * w[i] * t) * cqq_kubo))

    
plt.title('Comparison of the Fourier transfors of the \n standard and kubo-transformed autocorrelation functions')
plt.plot(w, kqq_std, label='std')
plt.plot(w, kqq_kubo, label='kubo')
plt.xlabel('$\omega$ [a.u.]')
plt.ylabel('Kqq($\omega)$ [a.u.]')
plt.legend()
plt.show()


plt.title('Verification of the detail balance relation')
plt.plot(w, kqq_kubo, label='kubo')
plt.plot(w, kqq_times_dbcf, label='std x harmonic detailed balance factor', ls='--')
plt.xlabel('$\omega$ [a.u.]')
plt.ylabel('Kqq($\omega)$ [a.u.]')
plt.legend()
plt.show()

# EXERCISE 2: Properties of classical dynamics

A harmonic oscillator (in reduced units) samples the following canonical distribution

$ P(p) \propto e^{-\beta p^2 / 2}$

$ P(q) \propto e^{-\beta q^2 / 2}$ 

Let us sample initial configurations from these distibutions. 

In [ ]:
n_trajs = 10

p0s = []
q0s = []

for i in range(n_trajs):
    p0s.append(np.random.normal())
    q0s.append(np.random.normal())

Let's also define the force on a harmonic oscillator (needed to perform molecular dynamics)

In [ ]:
def f(q):
    """
    The force acting on a simple harmonic oscillator [in reduced units]
    """
    return -q

Here is a simple velocity verlet scheme to perform molecular dynamics. The trajectories are saved in the dictionary `traj_ens`.

In [ ]:
traj_ens = {}

for j, p0, q0 in zip(range(n_trajs), p0s, q0s):

    p = p0
    q = q0
    dt = 0.01
    tmax = 100
    steps = int(tmax / dt)
    
    traj_ens[j] = {}

    plist = []
    qlist = []
    flist = []
    
    for i in range(steps):

        p = p + (f(q)) * dt / 2
        q = q + p * dt
        p = p + (f(q)) * dt / 2

        plist.append(p)
        qlist.append(q)
        flist.append(f(q))


    traj_ens[j]['plist'] = plist
    traj_ens[j]['qlist'] = qlist
    traj_ens[j]['flist'] = flist
    

## Conservation of phase-space volume

We note that the velocity-Verlet scheme can be summarized as follows:

$q(t + dt) = q(t) + p(t)~dt  + \frac{1}{2}~f(q(t))~dt^2$

$p(t + dt) = p(t) + \frac{1}{2} \left(f(q(t)) + f(q(t + dt))\right)~dt$

The Jacobian of the transformation of the phase-space vector from time $t$ to $t + dt$
can be calculated as

$J =  |\frac{\partial p(t + dt)}{\partial p(t)}  \frac{\partial p(t + dt)}{\partial q(t)} |$

$~~~~~~~  |\frac{\partial q(t + dt)}{\partial p(t)}  \frac{\partial q(t + dt)}{\partial q(t)} |$

$|~1 \quad \frac{1}{2} f'(q(t))~dt |$

$|dt \quad  1 + \frac{1}{2} f'(q(t))~dt^2 |$

Clearly, the determinant is 1, hence the dynamics conserves the differential volume of the phase-space vector. We can, however, verify the conservation of volume numerically in the following way.

Let us visualize the positions as a function of time for the sampled trajectories

In [ ]:
plt.title('Trajectories with different offsets')

for i in range(n_trajs):
    qlist = traj_ens[i]['qlist']
    dq = np.asarray(qlist)
    plt.plot(range(len(qlist))[i * int(len(qlist) / 20):], dq[i * int(len(qlist) / 20):], lw=1.0, ls='-', label='traj ' + str(i))
    plt.xlabel('step')
    plt.ylabel('q [a.u.]')

plt.xlim(0, int(len(qlist) * 1.3))
plt.legend()
plt.show()

Let us also visualize the phase-space vector a function of time

In [ ]:
plt.title('Trajectories')

for i in range(n_trajs):
    qlist = traj_ens[i]['qlist']
    plist = traj_ens[i]['plist']
    dq = np.asarray(qlist)
    plt.plot(qlist, plist, lw=1.0, ls='--', label='traj ' + str(i))
    plt.xlabel('step')
    plt.ylabel('q [a.u.]')

plt.legend()

Considering the phase-space vector to be a parametric curve (with respect to time), conservation of the phase-space volume can be numerically checked by calculating the arc length $\delta p^2 + \delta q^2$

In [ ]:
plt.title('Trajectories')

for i in range(n_trajs):
    qlist = traj_ens[i]['qlist']
    plist = traj_ens[i]['plist']
    flist = traj_ens[i]['flist']
        
    dq = np.asarray(plist) * dt
    dp = np.asarray(flist) * dt
        
    dq = np.asarray(plist[1:-1]) * dt + np.asarray(flist[1:-1]) * dt**2
    dp = 0.5 * (np.asarray(flist[1:-1]) + np.asarray(flist[2:])) * dt
    plt.plot(np.sqrt(dp**2 + dq**2) / dt, lw=1.0, ls='--', label='traj ' + str(i))
    plt.xlabel('step')
    plt.ylabel('q [a.u.]')
    plt.xlim(0, int(len(qlist) * 1.3))


plt.legend()
plt.show()

## Conservation of the Boltzmann distribution

To verify the conservation of the Boltzmann distribution let us calculate the time evolution of the Boltzmann weight $e^{-\beta E}$ where $E$ is the total energy.

In [ ]:
for j in range(n_trajs):
    ps = np.asarray(traj_ens[j]['plist'])
    qs = np.asarray(traj_ens[j]['qlist'])
    plt.plot(np.exp(-beta * (ps**2 / 2 + qs**2 / 2)))
    plt.yscale('log')
    plt.xlabel('step')
    plt.ylabel('Exp[-$E / (k_B T)$]')

## Time reversibility

To verify time reversibility let's select the negative of the final momenta and the final positions of the previously sampled trajectories as initial conditions and perform molecular dynamics. 

In [ ]:
inv_traj_ens = {}

for j in range(n_trajs):

    p = -traj_ens[j]['plist'][-1]
    q = traj_ens[j]['qlist'][-1]
    
    inv_traj_ens[j] = {}

    plist = []
    qlist = []
    flist = []
    
    for i in range(steps):

        p = p + (f(q)) * dt / 2
        q = q + p * dt
        p = p + (f(q)) * dt / 2

        plist.append(p)
        qlist.append(q)
        flist.append(f(q))


    inv_traj_ens[j]['plist'] = plist
    inv_traj_ens[j]['qlist'] = qlist
    inv_traj_ens[j]['flist'] = flist
    

To visualize time reversibility let us simultaneously plot the trajectory and the reverse of one initialized using the negative final momentum and the final position

In [ ]:
plt.plot(traj_ens[j]['qlist'])
plt.plot(inv_traj_ens[j]['qlist'][::-1], ls='--')

## The vibrational spectrum of a molecule of H$_{2}$O

In this exercise we will study a simple system that most of you must have been familiar with in your day-to-day lives -- a water molecule. You all may have also heard that there are three types of chracteristic collective motion in a molecule of water: symmetric and asymmetric stretching and a bending mode. Let us calculate the vibrational spectrum of water and check the efficiency of path-integral methods for its estimation.

PLACEHOLDER FIGURE

We will first go to the appropriate folder, create a separate working directory, copy the input_template.xml file and and edit it. You will need to open a terminal and execute

```
$ cd pimd-mooc/3-trpmd
$ mkdir T-300K
$ cd T-300K
$ mkdir P.001
$ cd P.001
$ cp ../../template_trpmd.xml input.xml
```

Set the number of beads `nbeads` to 1 and the temperature to 300 K. These settings will correspond to a standard classical molec ular dyanmics run with a weak thermostat and will give us the classical vibrational density of states. To run this simulation

```
$ i-pi input.xml > log.i-pi &
$ sleep 10
$ i-pi-driver -u -h h2o-molecule -m qtip4pf & 
```

In the meantime, create directories 

```
P.002 P.004 P.008 P.016 P.032 P.064 P.128
```

Copy the input.xml file from the root directory of exercise 3 to the inidividual directories and set the number of beads to 2, 4, 8, 16, 32, 64 and 128, and the temperature to 300 K, in the respective dirctories. Run these simulations (and remember to change the address of the UNIX socket ;))

To calculate the vibrational density of states run the following script

```
$ cd P.001
$ i-pi-getacf -ifile simulation.vc.xyz -mlag 1024 -ftpad 3072 -ftwin cosine-hanning -dt "0.25 femtosecond" -oprefix vv
$ cd ..
```

This will create files 

```
vv_acf.data and vv_facf.data
```

with the normalized velocity autocorrelation function and the the vibrational densitiy of states, respectively. Try the same for all the directories and study the convergence of the vibrational density of states with the number of beads.

In [ ]:
Ps = [1, 2, 4, 8, 16, 32]

plt.title('Convergence of the vibrational density of states\n of a water molecule at 300 K')
plt.xlim(500, 5000)
plt.ylim(-0.0002, 0.0006)
plt.xlabel(r'$\omega [cm^{-1}]$')
plt.ylabel(r'C_{vv} [a.u.]')
plt.grid()
for i, P in enumerate(Ps):
    
    w, cvv, cvv_err = np.loadtxt('3-rpmd/T-300K/P.' + str(P).zfill(3) + '/vv_facf.data').T
    cvv += 0.00005 * i
    w = w * 219474 # converts the frequency from atomic units to invcm
    plt.plot(w, cvv, alpha =1.0, ls = '--', lw=0.5, c='k')
    plt.errorbar(w, cvv, cvv_err, barsabove=True, label='P = ' + str(P), alpha =0.5)
plt.legend(ncol=3)

Can you identify the vibrational modes? Now having studied the covergence for a water molecule, can you also calculate the vibrational density fo states for liquid water at 300 K? Compare the two spectra and try to rationalize the difference in the stretching band. 

Replace ?? with the number of beads that should give a converged spectrum

```
$ mkdir T-300K-bulk
$ cd T-300K
$ mkdir P.0??
$ cd P.0??
$ cp ../../template_trpmd.xml input.xml
```

In [ ]:
plt.title('Vibrational density of states of water')
plt.xlim(50, 5000)
plt.ylim(0.0, 0.0002)
plt.xlabel(r'$\omega [cm^{-1}]$')
plt.ylabel(r'$C_{vv}$ [a.u.]')
plt.grid()

w, cvv, cvv_err = np.loadtxt('3-rpmd/T-300K/P.016/vv_facf.data').T
w = w * 219474 # converts the frequency from atomic units to invcm
plt.plot(w, cvv, alpha =1.0, ls = '--', lw=0.5, c='k')
plt.errorbar(w, cvv, cvv_err, barsabove=True, label='gas phase', alpha =0.5)

w, cvv, cvv_err = np.loadtxt('3-rpmd/T-300K-bulk/P.016/vv_facf.data').T
w = w * 219474 # converts the frequency from atomic units to invcm
plt.plot(w, cvv, alpha =1.0, ls = '--', lw=0.5, c='k')
plt.errorbar(w, cvv, cvv_err, barsabove=True, label='liquid phase', alpha =0.5)



plt.legend(ncol=3)